# Create a simulator that reports the machine's sensor values

We don't have a real machine to monitor, so let's simulate one.
For this we need a few components:

* a machine that keeps running (M1)
* a sensor attached to that machine (S1)
* an API that we can query from our application (M-API)

# Tasks

1. Run the code below
2. Query the machine using your web browser
3. Try to run different configurations and see if this changes your output

# The machine and it's sensor

In [ ]:
import numpy as np
from threading import Thread
from datetime import datetime as dt
from time import sleep

# specifcy the operation mode
# -- every regime is a tuple of (shape, scale)
REGIMES = {
     # Configuration 1: working ok, rare failure
    'ok': (1., .1),
    # Configuration 2: fail often
    'fail': (5., .1),
    # Configuration 3: constantly failing
    'error': (10., .1),
}

# sensor buffer
size = 1000
STATE = [(None, 0)] * size
SELECTED_REGIME = 'ok'

def sensor():
    global STATE
    rnd = np.random.default_rng()
    while True:
        # generate the data for the next <size> sensor values
        # -- according to the currently selected regime
        # -- remember the regime so we can detect a change 
        shape, scale = REGIMES[SELECTED_REGIME] 
        data = rnd.gamma(shape, scale, size)
        my_regime = SELECTED_REGIME
        # run through all the sensor values
        # -- wait .1 seconds between each new value (generate 10 values / seconds)
        # -- if the regime has changed, reset to generate new values
        # -- add a timestamp to every value
        for i, value in enumerate(data):
            record = (dt.now().isoformat(), value)
            STATE = STATE[1:] + [record]
            sleep(.1)
            if my_regime != SELECTED_REGIME:
                break

def machine():
    # we use a Thread to run the machine in the background
    # this way we get fresh sensor values continously
    t = Thread(target=sensor)
    t.start()

# The sensor API

The sensor API is very simple

* `http://localhost:5000/query/10` reports the latest 10 sensor values
* `http://localhost:5000/query/100` reports the latest 100 sensor values
* `http://locahost:5000/regime/` shows the currently active and the available regimes
* `http://locahost:5000/regime/<ok|fail|error>` changes the regime 
* it can report at most `size=1000` values (we could change this)

In [ ]:
from flask import Flask

def create_app():
    app = Flask("factory")

    @app.route("/")
    def index():
        index.__doc__ = f"""
            * /query/NNN: returns the latest NNN sensor values<br>
            * /regime/: returns the current sensor regime<br>
            * /regime/RRR: set the current sensor regime. RRR should be one of {REGIMES.keys()}<br>
        """
        return index.__doc__
    
    @app.route("/query/<int:records>")
    def get_data(records):
        values = STATE[-records:]
        return { 'values': values }
    
    @app.route("/regime/<regime>")
    @app.route("/regime/")
    def set_regime(regime=None):
        global SELECTED_REGIME
        if regime:
            SELECTED_REGIME = regime
        return { 'regime': SELECTED_REGIME ,'available': list(REGIMES.keys()) }
    
    return app

# start the machine (runs in the background)
machine()

# start the web app 
app = create_app()
app.run(port=5001)